# Python科学计算神器Numpy-5：内存结构
                                                —— By Li Michael

## 一. Array的内存结构
结构数组时提到内存对齐，这节让我们看看ndarray数组对象是如何在内存中存储的。关于Array的描述信息保存在一个数据结构中，这个结构引用两个对象：一个用于保存数据的存储区域，一个用于描述元素类型的dtype.

数据存储区域保存着数组中所有元素的二进制数据，dtype对象则知道如何将元素的二进制数据转换维可用的值. 数组的维数、大小等信息都保存在ndarray数组对象的数据结构中. 图中显示的是如下数组的内存结构：

In [2]:
import numpy as np
a = np.array([[0,1,2], [3,4,5], [6,7,8]], dtype=np.float32)

![numpy_memory_struct.png](numpy_memory_struct.png)
一个标准的array的内存结构共有5部分：
* dtype: array的数据类型，这也说明了为什么同一array不能包含多种数据类型，因此需要结构数组（structure array）
* dim count: array的维度个数
* dimensions: array每个维度的元素个数，即array的大小，dimensions的数组长度就是dim count
* strides: array的每个轴下标增加1时，数据存储区偏移的字节数
* data: array的数据区

## 二. strides和数据共享
strides中保存的是当每个轴的下标增加1时，数据存储区中的指针所增加的字节数. 例如图中的strides为12,4，即第0轴的下标增加1时，数据的地址增加12个字节：即a[1,0]的地址比a[0,0]的地址要高12个字节，正好是3个单精度浮点数的总字节数; 第1轴下标增加1时，数据的地址增加4个字节，正好是单精度浮点数的字节数.

如果strides中的数值正好和对应轴所占据的字节数相同的话，那么数据在内存中是连续存储的. 然而数据并不一定都是连续储存的，前面介绍过通过下标范围得到新的数组是原始数组的视图，即它和原始视图共享数据存储区域：

In [3]:
b = a[::2,::2]
b

array([[ 0.,  2.],
       [ 6.,  8.]], dtype=float32)

In [4]:
b.strides

(24, 8)

这里b的strides为什么不是(8,4)呢？这是因为数组b和a共享数据存储区，而b中的第0轴和第1轴都是数组a中隔一个元素取一个，因此数组b的srides变成了(24,8)，正好是数组a的两倍. 对照前面的图很容易看出数据0和2的地址相差8个字节，而0和6的地址相差24个字节.

元素在数据存储区的排列方式有两种：C语言格式和Fortran语言格式. 在C语言中，多维数组的第0轴是最上位的，即第0轴的下标增加1时，元素的地址增加的字节数最多; 而fortran语言的多维数组的第0轴是最下位的，即第0轴的下标增加1时，地址只增加一个元素的字节数. 在Numpy中，元素默认是以C语言格式排列存储的，如果希望改为Fortran格式，可以给数组传递order=“F”参数：

In [8]:
c = np.array([[0,1,2],[3,4,5],[6,7,8]], dtype=np.float32, order="F")
c.strides

(4, 12)